In [1]:
import os
import instructor
from openai import OpenAI
from dotenv import load_dotenv
from src.services.core.messages_service import MessagesService

load_dotenv()
client = instructor.patch(OpenAI(api_key=os.getenv("OPENAI_API_KEY")))

In [2]:
# Initialize vars
messages_service = MessagesService()
openai_model = "gpt-3.5-turbo"
user_interest_enum = None

# Initial system message
system_prompt = """You are an assistant that helps users find sushi restaurants and parking spots in Munich. 
When a user expresses interest in either sushi, parking, or both:
1. Use the set_user_interest function to record their preference
2. Provide relevant parking or restaurant information
3. ALWAYS respond with a natural language message summarizing the information for the user
"""

messages_service.add_system_message(system_prompt)

user_prompt = "Hey" # Also responds to normal messages
user_prompt = "Tell a joke"
user_prompt = "I want to eat sushi"
user_prompt = "I want to park my car"
user_prompt = "List all parking spots"
user_prompt = "How can i pay for parking?"
user_prompt = "Where can i get the cheapest sushi nearby?"
# user_prompt = input("eg. I am looking for a Sushi Restaurant in Munich.")

messages_service.add_user_message(user_prompt)
messages_service.print_messages()

{'role': 'system', 'content': 'You are an assistant that helps users find sushi restaurants and parking spots in Munich. \nWhen a user expresses interest in either sushi, parking, or both:\n1. Use the set_user_interest function to record their preference\n2. Provide relevant parking or restaurant information\n3. ALWAYS respond with a natural language message summarizing the information for the user\n'}
{'role': 'user', 'content': 'Where can i get the cheapest sushi nearby?'}


### Choose user interest
Based on user prompt, determine if user is interested in Chatting, Parking or Sushi

In [3]:
from src.models.intent.user_interest_intent import UserInterestIntent

# Get user interest
intent : UserInterestIntent = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=UserInterestIntent,
)

messages_service.add_function_message(UserInterestIntent.__name__, intent.interest)

user_interest_enum = intent.interest

messages_service.print_messages()

{'role': 'system', 'content': 'You are an assistant that helps users find sushi restaurants and parking spots in Munich. \nWhen a user expresses interest in either sushi, parking, or both:\n1. Use the set_user_interest function to record their preference\n2. Provide relevant parking or restaurant information\n3. ALWAYS respond with a natural language message summarizing the information for the user\n'}
{'role': 'user', 'content': 'Where can i get the cheapest sushi nearby?'}
{'role': 'function', 'name': 'UserInterestIntent', 'content': <UserInterestEnum.SUSHI: 'SUSHI'>}


### Choose response format based on user intend and handle intent
Its either Parking or Sushi

In [4]:
from src.services.user.user_interest_service import get_user_interest

response_model, intent_service = get_user_interest(user_interest_enum)

intent = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=response_model,
)

messages_service.print_last_message()
messages_service.add_function_message(intent.__class__.__name__, intent)
messages_service.print_last_message()

if intent_service is not None:
    result = intent_service.handle_intent(intent)
    messages_service.add_function_message(intent_service.handle_intent.__name__, result)
    messages_service.print_last_message()

{'role': 'function', 'name': 'UserInterestIntent', 'content': <UserInterestEnum.SUSHI: 'SUSHI'>}
{'role': 'function', 'name': 'UserSushiIntent', 'content': '{"query_types":["nearby","price_check"],"specific_restaurant_name":null,"location":null,"response_detail":"brief"}'}
{'role': 'function', 'name': 'handle_intent', 'content': '{"data":{"price":[{"title":"Sasou","data":{"priceRangeLevel":"3 out of 5","free":false}},{"title":"Galeria Restaurant","data":{"priceRangeLevel":"1 out of 5","free":false}},{"title":"Shaokao Asian Grill&Wine","data":{"priceRangeLevel":null,"free":false}},{"title":"Secret Garden","data":{"priceRangeLevel":null,"free":false}}],"nearby":[{"title":"Sasou","data":{"distance_from_current_location":"65.29277618006775 meters","duration_from_current_location":null}},{"title":"Galeria Restaurant","data":{"distance_from_current_location":"200 meters","duration_from_current_location":"1 minute"}},{"title":"Shaokao Asian Grill&Wine","data":{"distance_from_current_location"

### Generate final response
Summarize the result and return it as a natural language message

In [5]:
from src.models.intent.assistant_model import AssistantModel

response : AssistantModel = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=AssistantModel,
)

messages_service.add_assistant_message(response.assistant_message)
messages_service.print_last_message()

{'role': 'assistant', 'content': 'The cheapest sushi nearby can be found at Galeria Restaurant. It is located 200 meters away, around a 1-minute walk from your current location.'}


### Use it like a chat

In [6]:
from src.models.intent.user_interest_intent import UserInterestIntent

for i in range(2):
    
    # get user input
    user_prompt = input("User: ")
    messages_service.add_user_message(user_prompt)
    messages_service.print_last_message()

    # Get user interest
    interest_intent: UserInterestIntent = client.chat.completions.create(
        model=openai_model,
        messages=messages_service.get_messages(),
        response_model=UserInterestIntent,
    )

    messages_service.add_function_message(UserInterestIntent.__name__, interest_intent.interest)
    messages_service.print_last_message()
    
    # Get appropriate response model and service based on user interest
    response_model, intent_service = get_user_interest(interest_intent.interest)

    # Get specific intent (sushi or parking or chatting)
    intent = client.chat.completions.create(
        model=openai_model,
        messages=messages_service.get_messages(),
        response_model=response_model,
    )


    # If not a assistance intent, handle intent and get result
    if intent_service is not None:
        messages_service.add_function_message(intent.__class__.__name__, intent)
        messages_service.print_last_message()
        result = intent_service.handle_intent(intent)
        messages_service.add_function_message(intent_service.handle_intent.__name__, result)
        messages_service.print_last_message()

        # llm generates final response based on intent and result
        response = client.chat.completions.create(
            model=openai_model,
            messages=messages_service.get_messages(),
            response_model=AssistantModel,
        )

        messages_service.add_assistant_message(response.assistant_message)
        messages_service.print_last_message()
        continue
    
    messages_service.add_assistant_message(intent.assistant_message)
    messages_service.print_last_message()


{'role': 'user', 'content': 'nice, where can i park cheapest?'}
{'role': 'function', 'name': 'UserInterestIntent', 'content': <UserInterestEnum.PARKING: 'PARKING'>}
{'role': 'function', 'name': 'UserParkingIntent', 'content': '{"query_types":["cheapest","nearby"],"specific_spot_name":null,"duration_hours":null,"height_requirement":null,"location":null,"needs_disabled_access":null,"payment_method_preference":null,"response_detail":"brief"}'}
{'role': 'function', 'name': 'handle_intent', 'content': '{"data":{"cheapest":["City",{"priceSummaryText":"6.00 € / 2h","free":false},{"listPrices":[{"service":"PARKING","price":"Open 24 hours: 3.00 € / 1 hour"},{"service":"PARKING","price":"Open 24 hours: 6.00 € / 2 hours"},{"service":"PARKING","price":"Open 24 hours: 9.00 € / 3 hours"},{"service":"PARKING","price":"Open 24 hours: 12.00 € / 4 hours"},{"service":"PARKING","price":"Open 24 hours: 15.00 € / 5 hours"},{"service":"PARKING","price":"Open 24 hours: 18.00 € / 6 hours"},{"service":"PARKING"

In [7]:
# reset jupiter kernel
# %reset -f